In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 7.5 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch import nn

In [7]:
from google.colab import drive
drive.mount('/content/drive')
os.listdir("/content/drive/My Drive/3조")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['DS_folder',
 '1.기획안&WBS',
 '2.수행일지',
 '3.최종포트폴리오',
 '4.과제',
 '멘토링',
 '회의록',
 '[D11&12]기업요구사항 기반의 문제해결 프로젝트.pdf',
 'Data',
 '빅데이터분석프로젝트_단계별공정.pdf',
 'DE']

In [8]:
PATH = '/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/'

In [28]:
test = pd.read_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/song/songChart.csv', index_col=0)

In [13]:
test = test['lyrics']

In [14]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizerFast'.


In [18]:
titles_t = test.to_numpy().reshape(-1,1)


# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
test = pd.DataFrame({'title':titles_t.reshape(-1)})

In [19]:

# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )
  


In [20]:
# 데이터 로더 생성
BATCH_SIZE =64
MAX_LEN =32
# train_data_loader = create_data_loader(train, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader = create_data_loader(test, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
import random
class NewsSubjectClassifier(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier, self).__init__()
    self.bert = AlbertModel.from_pretrained("kykim/albert-kor-base")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

device = torch.device("cpu")

def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs


In [22]:
model = torch.load(PATH+"model_albert_kor_base.pth")


In [23]:
subject_texts, predictions, prediction_probs = get_predictions(model, valid_data_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [44]:
ls = []
for i in prediction_probs:
  ls.append(i[:6])
len(ls)

1029

In [45]:
df = pd.DataFrame({'story':subject_texts, 'story_predict':predictions, 'predict_probs':ls})

In [47]:
df.to_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/song_감성분류.csv')

In [37]:
test['lyrics'][0]

'처음이라 그래 며칠뒤엔 괜찮아져 그 생각만으로 벌써 일년이 너와 만든 기념일마다 슬픔은 나를 찾아와 처음 사랑 고백하며 설렌 수줍음과 우리 처음 만난날 지나가고 너의 생일엔 눈물의 케이크 촛불 켜고서 축하해 I believe in you I believe in your mind 벌써 일년이 지났지만 일년뒤에도 그 일년뒤에도 널 기다려 너무 보고싶어 돌아와줘 말 못했어 널 보는 따뜻한 그의 눈빛과 니 왼손에 껴진 반지보다 빛난 니 얼굴 때문에 I believe in you I believe in your mind 다시 시작한 널 알면서 이젠 나없이 추억을 만드는 너라는걸 내가 기억하는 추억은 언제나 지난 웃음과 얘기와 바램들 또 새로 만들 추억은 하나뿐 내 기다림과 눈물 속 너일뿐 I believe in you I believe in your mind 다시 시작한 널 알면서 이젠 나없이 추억을 만드는 너라는걸 I believe in you I believe in your mind 벌써 일년이 지났지만 일년뒤에도 그 일년뒤에도 널 기다려'

In [38]:
prediction_probs[0][:6]

tensor([2.5198, 1.9887, 5.9538, 2.0492, 3.4132, 2.5087])